Here is the template for data science project, this notebook focus exclusively on exploratory data analysis part:

By Emma HONG, DSBA master student from CentraleSuperlec

# Ordinary EDA

In [ ]:
# import the necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime

# for modeling
import statsmodels.api as sm
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer

from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.decomposition import PCA

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from skopt import BayesSearchCV
from sklearn.model_selection import KFold
import joblib

# for scoring
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error # regression
from sklearn.metrics import confusion_matrix, roc_curve, f1_score, accuracy_score, hinge_loss

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pylab import rcParams
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
# Connect to Google Drive for later use
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Show the files in the root filefolder
import os
for dirname, _, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Basic

In [ ]:
# Read data from files
path = ''
headnum = 1 # Change it according to the real case
separater = ',' # Sometimes it's ';'
# CSV file
df = pd.read_csv(path,sep=separater) # add this code when you don't want certain columns: usecols = lambda x: x not in ['', ''] 

# Json file
df = pd.read_json(path)
# If you want to turn json file into a 
key = '' # Column name for the key
value = '' # Column name for the value
dictionary = df.set_index(key)[value].to_dict()

# Excel file
df = pd.read_excel(path,header = headnum)

In [ ]:
# Specifically, if you want to merge the dataframe based on same columns
df = pd.concat([df1,df2], axis = 0) # add this when you only want to merge same columns: join='inner'
# or use methods similar to sql
df_merge = df.merge(df1, how ='left', right_on='right_id', left_on='left_id')

In [ ]:
# Make a copy if the later processing will overwrite the dataframe
df_copy = df.copy()

In [ ]:
# Here is the codes to download data from Kaggle
! pip install kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json # Get your kaggle.json from Kaggle website in advance

# Import a toy kaggle dataset to try the model interaction
!pip install opendatasets
import opendatasets as od
kagglepath = '' # The link of the target file page
od.download(kagglepath)

In [ ]:
# Take the first look on the data
df.head() # You can specify the rows to view here

In [ ]:
# To be more clearer, show the meta information of each column
df.info()

In [ ]:
# Generate descriptive statistics
df.describe()

In [ ]:
# If you want to adjust the format of some columns manually
# Encode the time format column
columnname = ''
typename = % # object, int64, float64, bool, datetime64, timedelta, category
df[columnname] = pd.to_datetime(df[columnname])
# Change the type for a specific column
df[columnname] = df[columnname].astype(typename)

In [ ]:
# number of rows for each column
df.count()

In [ ]:
# Check the number of rows contain na value
df.isnull().sum()

In [ ]:
# Show the number of missing value and frequency

missing_values = df.isna().sum()

df_missing = pd.DataFrame(data = missing_values, columns = ['count'], index = dict(missing_values).keys())
df_missing['frequency'] = missing_values / df.shape[0]
df_missing = df_missing.sort_values(by = 'count', ascending = False)

df_missing.head(20)

In [ ]:
# check if the dataframe has duplicated rows(can limit to specific columns if you want)
df.duplicated().sum()

## Distribution

In [ ]:
# Plot the distribution of the values in certain column (categorical columns)
cat_features = ['','','']

# Create a figure with subplots
# May need to change the nrows and ncols to suit the specific case
fig, axes = plt.subplots(nrows=1, ncols=len(cat_features), figsize=(len(cat_features)*6, 6))

# Loop through each categorical column and plot its value counts on a subplot
for i, columnname in enumerate(cat_features):
    df[columnname].value_counts().plot(kind='bar', ax=axes[i]) #kind = 'barh' if you prefer horizontal one
    axes[i].set_title(columnname)

# Adjust spacing between subplots and show the figure
plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
plt.show()


In [ ]:
# Another way to plot the distribution of caterical data(The input df should only contain categorical columns)
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k') # white/black
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            # To increase the readibility, only show top 50 results here
            valueCounts = columnDf.value_counts().head(50)
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
# Plot the distribution of the values in certain column (numerical columns)
num_features = ['','','']
df[list_num_features].hist(bins = 100, figsize = (30, 25));

In [ ]:
# Another way to show the variable distribution
optimal_bins_target = round(math.log(df.shape[0], 2) + 1) # based on Sturge's Rule: Optimal Bins = ⌈log_2(n) + 1⌉
df[columnname].hist(bins = optimal_bins_target)

# Print the skewness and kurtosis value
print("Skewness: %.2f" % df['valeurfonc'].skew())
print("Kurtosis: %.2f" % df['valeurfonc'].kurt())

In [ ]:
# check distribution of numerical value
fig, axes = plt.subplots(1,3, figsize=(21,6))
sns.distplot(df[columnname], ax=axes[0])
sns.distplot(np.log1p(df[columnname]), ax=axes[1])
axes[1].set_xlabel('log(1+price)')
# QQ plot
sm.qqplot(np.log1p(df[columnname]), stats.norm, fit=True, line='45', ax=axes[2])
plt.show()

In [ ]:
# calculation about numerical column
pd.set_option('display.float_format',lambda x : '%.2f' % x) # Cancel scientific notation

groupcolumn = ''
result = df.groupby(groupcolumn).agg({columnname: ['mean', 'min', 'max']})
print("Mean, min, and max values of price_per_square grouped by groupcolumn")
print(result)

## Correlation

In [ ]:
# Relationship between numerical data
numericallist = ['','','']
df_num = df[numericallist]
# Only Sample 500 to get the intuition, use these codes when the dataset has large volume
df_numsome = df_num.sample(n=500)
sns.pairplot(df_numsome)

In [ ]:
# When the data distribution has extreme outliers
for col in numericallist:
    df_numsome[col] = np.log(df_numsome[col])
sns.pairplot(df_numsome)

In [ ]:
# Define functions to remove the outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] > lower_bound) & (df[column] < upper_bound)]
df_ro = remove_outliers(df,columnname)

In [ ]:
# Show the distibution after removing the outliers

def without_outlier(df, columnname):
    # Calculate interquartile range of the prices
    q25 = np.percentile(df[columnname].values, 25)
    q75 = np.percentile(df[columnname].values, 75)
    IQR = q75 - q25

    # Calculate the outlier cutoff
    lower_bound = q25 - IQR*1.5
    upper_bound = q75 + IQR*1.5

    # Identify outliers and remove them for the data visualization
    fig = plt.figure(figsize = (11, 7))
    ax = fig.gca()
    valueinside = df.loc[(df[columnname] >= lower_bound) & (df[columnname] <= upper_bound)][columnname]
    valueinside.hist(bins = 100, ax = ax)
    plt.show()

    # Display the Skewness and Kurtosis of the prices without the outliers
    print("Skewness: %.2f" % valueinside.skew())
    print("Kurtosis: %.2f" % valueinside.kurt())

In [ ]:
# Correlation heatmap(prettier, show only half of the graph)
# Change the style theme
sns.set(style = "white")

non_numeric_columns = ['','','']
# Compute the correlation matrix
corr = df.drop(non_numeric_columns, axis = 1).corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype = bool))

# Set up the matplotlib figure
fig, axes = plt.subplots(figsize = (11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap = True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask = mask, cmap = cmap, center = 0,
            square = True, linewidths = .5, cbar_kws = {"shrink": .5}); # cbar_kws: colorbar setting

In [ ]:
# Show the top 15 correlated features to the dependent variable
targetcolumn = ''
correlated_features = corr[targetcolumn].sort_values(ascending = False)

correlated_features[1:16]

In [ ]:
# Store the top 15 in a new DataFrame

top_15_correlated_features = correlated_features[1:16].index

df_top15_correlated_features = df[top_15_correlated_features]

df_top15_correlated_features.head(10)

In [ ]:
# Check the correlation between the top 15 features(They may be highly correlated between each other)

# Compute the correlation matrix
corr_top15 = df[top_15_correlated_features].corr()

# Generate a mask for the upper triangle
mask_top15 = np.triu(np.ones_like(corr_top15, dtype = bool))

# Set up the matplotlib figure
fig, axes = plt.subplots(figsize = (11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_top15, mask = mask_top15, cmap = cmap, center = 0,
            square = True, linewidths = .5, cbar_kws = {"shrink": .5});

## Dealing with time series data

In [ ]:
datecolumn = ''
valuecolumn = ''
# The distribution of data by time ( For time series data)
datecolumn = ''
df[datecolumn] = pd.to_datetime(df[datecolumn], format='%m/%d/%Y') # The format of date can change
df[datecolumn] = df[datecolumn].dt.strftime('%Y-%m-%d')
df[datecolumn].value_counts().sort_index().plot(kind = 'line')

In [ ]:
# For panel data
# Creating another dataset so we can re-use the original if we need it
df_ = df.copy()
df_ = df.sort_values(by=[datecolumn])


In [ ]:
# resample the data by month
dfM = df_.resample('M').mean()
# check the boxplot of average price by month
dfM.loc[:,[valuecolumn]].boxplot()

In [ ]:
# show the price change with the time
fig, ax = plt.subplots(figsize=(12,6))
dfM[valuecolumn].plot()
plt.show()

In [ ]:
# filter other factor
factor = ''
xx = 10 # customized value
df_xx = df[df[factor]==xx]
df_xx_M = df_xx.resample('M').mean().dropna()
df_xx_Y = df_xx.resample('Y').mean()
fig, ax = plt.subplots(figsize=(12,6))
df_xx_M[valuecolumn].plot()
plt.title("average value change by month")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
df_xx_Y[valuecolumn].plot()
plt.title("average value change by year")
plt.show()

In [ ]:
# Check seasonality
y_m_avg = df_xx_M.groupby(datecolumn)[valuecolumn].mean()
y_m_avg = y_m_avg.dropna()

decomposition = sm.tsa.seasonal_decompose(y_m_avg, model='additive', period=6)

#Gather the trend, seasonality, and residuals
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plot gathered statistics
plt.figure(figsize=(12,8))
plt.subplot(411)
plt.plot(y_m_avg, label='Original', color='blue')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend', color='blue')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality', color='blue')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals', color='blue')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
# Stationary check

def stationarity_check(TS):
    
    # Import adfuller
    from statsmodels.tsa.stattools import adfuller
    
    # Calculate rolling statistics
    roll_mean = TS.rolling(window=8, center=False).mean()
    roll_std = TS.rolling(window=8, center=False).std()
    
    # Perform the Dickey Fuller test
    dftest = adfuller(TS) 
    
    # Plot rolling statistics:
    fig = plt.figure(figsize=(12,6))
    orig = plt.plot(TS, color='blue',label='Original')
    mean = plt.plot(roll_mean, color='red', label='Rolling Mean')
    std = plt.plot(roll_std, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    # Print Dickey-Fuller test results
    print('Results of Dickey-Fuller Test: \n')

    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', 
                                             '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
    return None

In [ ]:
ts_log_decompose = residual
ts_log_decompose.dropna(inplace=True)
stationarity_check(ts_log_decompose)

In [ ]:
# Check autocorrelation
# plot the ACF and PACF
rcParams['figure.figsize']=7,5
plot_acf(y_m_avg); plt.xlim(0,24); plt.show()
plot_pacf(y_m_avg); plt.xlim(0,24); plt.ylim(-1,1);plt.show()

## Using PySpark to deal with big data

In [ ]:
# Configuration codes
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar zxvf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark
 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"


import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext(conf = conf)
print("initialization successful!")


#import of the SparkSession
from pyspark.sql import SparkSession

#inizialization of the Spark Session
spark = SparkSession \
    .builder \
    .appName("Template") \
    .getOrCreate()

import numpy as np
import random as rn
import pandas as pd

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)


In [ ]:
filename = ''
# Load the file into an RDD
RDD = sc.textFile(filename)

# Create RDD with files, use wholeTextFiles to distinguish the files
def load_text(filename):
    return sc.wholeTextFiles(filename)
# Read the texts into RDD, you can read several files in a time
texts = load_text("filefolder/*.txt")

Data example: 

[('file:/content/the_office/33793.txt',
  "06x12 - Scott's Tots\nMikela\nMikela: We just want to say thanks.\n"),
 ('file:/content/the_office/4199.txt',
  '02x08 - Performance Review\nMichael\nMichael: What do you say, Jan?\n'),
 ('file:/content/the_office/11602.txt',
  "03x08 - The Merger\nPam\nPam: Oh, I'll just time him later.\n"),
 ('file:/content/the_office/6578.txt',
  '02x18 - Take Your Daughter to Work Day\nToby\nToby: [to Sasha] Okay, tell them what you wanted to say.\n'),
 ('file:/content/the_office/9020.txt',
  '02x99 - Deleted Scenes from Season 2\nMichael\nMichael: I used to be in HR. I was a Hell raiser.\n')]

In [ ]:
# Functions frequently used to deal with texts data

import string
import re
from stop_words import get_stop_words

# remove punctuations, only remove the punctuations on both sides
def remove_punctuation(word):
    punctuation = string.punctuation
    word = word.strip(punctuation)
    return word

# remove punctuations
def remove_non_letters(word):
    if type(word) != str:
        word = ""
    regex = re.compile(r'[^\w\s]')
    return regex.sub('', word)

# remove the stop words
def removestopwords(word):
    stop_words = get_stop_words('english')
    if word in stop_words:
      filtered_word = ""
    else:
      filtered_word = word
    return filtered_word

In [ ]:
# Sample codes for conducting MapReduce method on RDDs
# Use mapreduce to count the number of words in the whole set of dialogs
# Map method can apply the same function to the RDD element wise
diagnum = diag.map(lambda x: (x,1))\
              .reduceByKey(lambda x,y: x+y)\
              .sortBy(lambda f: f[1], ascending=False)

In [ ]:
# If you only want to apply the function on the value of a pair (x,y) within the RDD, use mapValues function
# Use case: Find common words by character
# Gather the dialogs under each character
cha_diag = texts.map(lambda x: (x[1].split("\n")[1],dialog(x[1].split("\n")[2]).split(" ")))\
                .reduceByKey(lambda x,y:x+y)
# Use mapValues to only process the value to split the words and clean the words
# The flatMap sentence is to spread the (character,word) and then make it easier to count the word frequency for each character
rdd = cha_diag.flatMap(lambda x: [(x[0], y) for y in x[1]])\
         .mapValues(lambda word: remove_punctuation(word))\
         .mapValues(lambda word: word.lower())\ # lower() method is useful to remove the influence of the capitalizartion of the words
         .mapValues(lambda x: removestopwords(x))\
         .filter(lambda word: len(word[1]) > 0) # empty string is not useful
# Do the mapreduce to get the result
# sorted sentence is to sort the tuples by the word frequency
rdd1 = rdd.map(lambda x: (x, 1))\
          .reduceByKey(lambda x, y: x+y)\
          .map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey()\
          .map(lambda x: (x[0], sorted(x[1], key=lambda y: y[1], reverse=True))) \
          .map(lambda x: (x[0],x[1][:10]))

# Here we only put 5 as examples
print("The 10 most common words for each character is:",rdd1.take(5))

In [ ]:
# Spark Dataframe, similar to pandas dataframe
# Read the csv series
dfs =spark.read.option("inferSchema", "true").option("delimiter", ",").csv("XXX.csv", header=True)
dfs.show(5)



```
# Example of data
+---+------+-------------+--------------------+-------+-----+----------+--------+--------------+----------+---------------+--------------------+
|_c0|Season| EpisodeTitle|               About|Ratings|Votes|Viewership|Duration|          Date|GuestStars|       Director|             Writers|
+---+------+-------------+--------------------+-------+-----+----------+--------+--------------+----------+---------------+--------------------+
|  0|   1.0|        Pilot|The premiere epis...|    7.5| 4936|      11.2|      23| 24 March 2005|      null|     Ken Kwapis|Ricky Gervais |St...|
|  1|   1.0|Diversity Day|Michael's off col...|    8.3| 4801|         6|      23| 29 March 2005|      null|     Ken Kwapis|         B. J. Novak|
|  2|   1.0|  Health Care|Michael leaves Dw...|    7.8| 4024|       5.8|      22|  5 April 2005|      null|Ken Whittingham|    Paul Lieberstein|
|  3|   1.0| The Alliance|Just for a laugh,...|    8.1| 3915|       5.4|      23| 12 April 2005|      null|   Bryan Gordon|       Michael Schur|
|  4|   1.0|   Basketball|Michael and his s...|    8.4| 4294|         5|      23| 19 April 2005|      null|   Greg Daniels|        Greg Daniels|
+---+------+-------------+--------------------+-------+-----+----------+--------+--------------+----------+---------------+--------------------+
only showing top 5 rows

```



In [ ]:
# Show 5 value as example
dfs_distinct = dfs.select(columnname).distinct()
dfs_distinct.show(5)

In [ ]:
# Find the number of episodes each director directed
# Using parquet to read dataframe and use pandas function to count the number of episodes each director directed
# Finally transform pandas dataframe into spark dataframe
dfs = dfs.select("Director","EpisodeTitle")
dfs.write.parquet("template/sample",mode='overwrite')
df = pd.read_parquet("template/sample")
# Use pandas dataframe to count
pdf = df["Director"].value_counts().sort_values(ascending=False)
pdf1 = [(i,v) for i,v in pdf.iteritems()]

# Finally tranform the pandas dataframe back to spark dataframe and print out the result
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([ \
    StructField("Director",StringType(),True), \
    StructField("number_of_episode",IntegerType(),True),
  ])
rdd = sc.parallelize(pdf1)
sdf = spark.createDataFrame(rdd,schema)
print("The number of episodes each director directed: ")
print(sdf.where("Director <> ''").show(5))

In [ ]:
# Find the 10 most common words for each character
import collections
# Read the data in pandas dataframe
df = pd.read_csv("XXX.csv",sep=';')

# Deal with the words
df["words"] = df["Field"].apply(lambda x: dialog(x)).apply(lambda x: x.split(" "))
df1 = df.explode("words") # Spread the words in rows
df1["words"] = df1["words"].apply(remove_punctuation) # Remove the punctuation in both sides
df1["words"] = df1["words"].apply(lambda x: x.lower())# Convert the words to lowercase
df1["words"] = df1["words"].apply(removestopwords)# Remove stopwords
df1 = df1[df1["words"] != ""] # Filter the empty elements
# Count the word frequency by characters and save the top 10 common words
top10_words = df1.groupby("Field1")
top10_words = top10_words.apply(lambda x: collections.Counter(x["words"]).most_common(10))
top10_words = pd.DataFrame(top10_words)

print("The 10 most common words for each character are:")
# Here we take 5 characters as examples:
top10_words.head(5)

In [ ]:
# Find the 10 episodes having the most number of words
from pyspark.sql.functions import explode, count,split,desc,lower,udf
# Select the dialogs and episodes to process
dfs = dfst.select("Field","Text")
dfs = dfs.withColumnRenamed("Text","episode")
# Count the words for each episode
udf_dialog = udf(dialog)
dfs = dfs.withColumn("dialogs",udf_dialog(col("Field"))) # extract the dialog
dfs = dfs.withColumn("words", explode(split("dialogs", " "))) # spread the words
word_counts = dfs.groupBy("episode").agg(count("words").alias("word_count")).orderBy(desc("word_count")) # Count the words by episode

print("The 10 episodes having the most number of words are:")
# Print the result:
word_counts.show(10)

# Data preprocessing

In [ ]:
# Do One-hot encoding to '',''
df_encoded = pd.get_dummies(df, columns=['',''])

In [ ]:
# Create scaler object
scaler = MinMaxScaler()

# indicate the numerical columns that need to be scaled
numeric_cols = ['','']

# log first(better change the column name, do it later)
for col in numeric_cols:
    df_encoded[col] = np.log(df_encoded[col]+1)

# Scale data
scaler.fit(df_encoded[numeric_cols])
scaler = scaler.transform(df_encoded[numeric_cols])
scaler = pd.DataFrame(scaler,columns=numeric_cols)
df_encoded[numeric_cols] = scaler

In [ ]:
df_encoded.columns

In [ ]:
# drop useless columns
keepcolumns = ['', '']
train_df = df_encoded[keepcolumns]
train_df = train_df.fillna(0)

In [ ]:
cols = train_df.columns.tolist()
cols.remove('')
X = train_df[cols]
y = train_df[['']]
# If PCA
#pca=PCA(n_components='mle')
#X = pca.fit_transform(X)
#pca1=PCA(n_components=33)
#test_df = pca1.fit_transform(test_df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Model training(including tuning part)

### Feature selection

In [ ]:
# Example from https://www.datacamp.com/tutorial/feature-selection-python
# load data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(url, names=names)

array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

In [ ]:
# Import the necessary libraries first
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

**Filter Method**

In [ ]:

# Feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)

# Summarize scores
np.set_printoptions(precision=3)
print(dataframe.columns)
print(fit.scores_)

features = fit.transform(X)
# Summarize selected features
print(features[0:5,:])

Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class'], dtype='object')
[ 111.52  1411.887   17.605   53.108 2175.565  127.669    5.393  181.304]
[[148.    0.   33.6  50. ]
 [ 85.    0.   26.6  31. ]
 [183.    0.   23.3  32. ]
 [ 89.   94.   28.1  21. ]
 [137.  168.   43.1  33. ]]


**Wrapper Method**

In [ ]:
# Import your necessary dependencies
# Recursive Feature Elimination 
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


In [ ]:
# Feature extraction
model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=3)
fit = rfe.fit(X, Y)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Num Features: 3
Selected Features: [ True False False False False  True  True False]
Feature Ranking: [1 2 4 5 6 1 1 3]


**Embedded Methods**

In [ ]:
# First things first
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X,Y)
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
# A helper method for pretty-printing the coefficients
def pretty_print_coefs(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)
    
print ("Ridge model:", pretty_print_coefs(ridge.coef_))

Ridge model: 0.021 * X0 + 0.006 * X1 + -0.002 * X2 + 0.0 * X3 + -0.0 * X4 + 0.013 * X5 + 0.145 * X6 + 0.003 * X7


## Unsupervised learning

### K-means

In [ ]:
# Unsupervised learning
# Elbow method to determine the number k of clusters

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

# Initialisation of the list of values k
number_clusters = range(1, 7)

# Initialisation of an empty list 'distortions' which will contain the distortion values of each Kmeans models depending of the number k
distortions = []

# For each value of k, fit a different KMeans algorithm
for k in number_clusters:
    kmeans = KMeans(n_clusters = k, random_state = 42)
    kmeans.fit(X_scaled)

    # Compute the distortion of each constructed model
    distortions.append(sum(np.min(cdist(X_scaled, kmeans.cluster_centers_, 'euclidean'), axis = 1)) / np.size(X_scaled, axis = 0))
    
# Create the distortions graph depending on the number of clusters
plt.figure(figsize = (16, 9))
plt.plot(number_clusters, distortions, 'gx-')
plt.xlabel('Number of Clusters k')
plt.ylabel('Distortion (WSS/TSS)')
plt.title('Elbow method displaying the optimal number of clusters')
plt.savefig('KMeans_distortions_graph.png')
plt.show()

Quote from : https://github.com/smazzanti/are_you_still_using_elbow_method/blob/main/are-you-still-using-elbow-method.ipynb

In [ ]:
import sklearn.metrics as sklearn_metrics

def calinski_harabasz_score(X, labels):
  """Wrapper function of Scikit-learn's calinski_harabasz_score. The only difference is it doesn't throw an error where there is only one label."""
  
  if len(set(labels)) == 1:
    return float("NaN")
  else:
    return sklearn_metrics.calinski_harabasz_score(X, labels)


def bic_score(X: np.ndarray, labels: np.array):
  """
  BIC score for the goodness of fit of clusters.
  This Python function is translated from the Golang implementation by the author of the paper. 
  The original code is available here: https://github.com/bobhancock/goxmeans/blob/a78e909e374c6f97ddd04a239658c7c5b7365e5c/km.go#L778
  """
    
  n_points = len(labels)
  n_clusters = len(set(labels))
  n_dimensions = X.shape[1]

  n_parameters = (n_clusters - 1) + (n_dimensions * n_clusters) + 1

  loglikelihood = 0
  for label_name in set(labels):
    X_cluster = X[labels == label_name]
    n_points_cluster = len(X_cluster)
    centroid = np.mean(X_cluster, axis=0)
    variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
    loglikelihood += \
      n_points_cluster * np.log(n_points_cluster) \
      - n_points_cluster * np.log(n_points) \
      - n_points_cluster * n_dimensions / 2 * np.log(2 * math.pi * variance) \
      - (n_points_cluster - 1) / 2
    
  bic = loglikelihood - (n_parameters / 2) * np.log(n_points)
        
  return bic

In [ ]:
# Training part
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

In [ ]:
# User case
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from matplotlib import pyplot

# load data
df = pd.read_csv('活跃用户分析.csv')

# substract
x = df[['年龄指数','参加活动指数','入会时间指数']].iloc[:,:].values

# Choose the best K
K = [1,2,3,4,5,6,7,8]
GSSE = []
for k in K:
    SSE = []
    kmeans = KMeans(n_clusters = k, random_state = 10)
    kmeans.fit(x)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    for label in set(labels):
        SSE.append(np.sum(np.sum(x[labels == label,]-centers[label,:]**2)))
    GSSE.append(np.sum(SSE))
    
# Plot the relationship between K and SSE(sum of squared errors)
plt.plot(K,GSSE,"b*-")
plt.xlabel("K")
plt.ylabel("SSE")
plt.title("Choose the best K")
plt.show()
    
# Define the model 
model = KMeans(n_clusters = 5)
# Model fit
model.fit(x)
# Assign a cluster to each instance
yhat = model.predict(x)
# Create a sample scatter graph
# plt.scatter(x[:,0],x[:,1],c = yhat,s= 50,cmap = 'viridis')

# plot 3D clustering results
from mpl_toolkits.mplot3d import Axes3D


ax = plt.subplot(111, projection='3d')  # Create a 3D drawing
# Divide the data points into three parts for drawing, with distinction in color
ax.scatter(x[:,0],x[:,1],x[:,2],c = yhat,s= 5,cmap = 'viridis')
centers = model.cluster_centers_

# Create a scatter graph of clustering results
ax.scatter(centers[:,0],centers[:,1],centers[:,2],c='red',s=100,alpha=0.5)

# Plot
pyplot.show()


## Classification tasks

### Supported Vector Machine (SVC)

In [ ]:
# Train with SVM - not effective enough compared to other models
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameter =  {'C': [0.1, 1, 10],
              'gamma': [0.1, 1, 10, 100],
              'kernel': ['rbf', 'linear']}
SVM_model = SVC(kernel = 'linear', C= 1)
#SVM_model = GridSearchCV(SVC(), parameter,verbose=2)
SVM_model.fit(training_features, y_train)
#print(SVM_model.best_params_)

y_train_pred = SVM_model.predict(training_features)
accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train accuracy: {accuracy}")

# Predict labels for validation set and evaluate performance
y_val_pred = SVM_model.predict(val_features)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy: {accuracy}")

# Predict labels for test set and generate submission file
y_test_pred = SVM_model.predict(testing_features)
submission = pd.DataFrame({"ID": range(test_data.shape[0]), "Predicted": y_test_pred})
submission.to_csv("submission_svm.csv", index=False)

### Multi-layer classifier (neural network)

In [ ]:
# 50,50 is the pair which gets best result 
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50,),max_iter = 50,verbose = True,random_state=1)
clf.fit(X, y)
# Model score
r = clf.score(X, y)
print("Accuracy:", r)
# Prediction
y_predict = clf.predict(test_df.fillna(0))  
y_predict = pd.DataFrame(y_predict, columns=['label'])
y_predict.to_csv("NN.csv", index=True, index_label='Id')

In [ ]:
# Tuning the model
parameter = {"hidden_layer_sizes": range(30,100,10),
              "max_iter": range(30,120,10)}
NN_model = MLPClassifier(verbose = True,solver= 'adam')
clf = RandomizedSearchCV(NN_model, parameter) # Can change the cv algorithm here, but randomizedsearchcv is faster here
clf.fit(X, y)
print(clf.best_params_)

In [ ]:
# Model score
r = clf.score(X, y)
print("Accuracy:", r)
# Prediction
y_predict = clf.predict(test_df.fillna(0))  
y_predict = pd.DataFrame(y_predict, columns=['label'])
y_predict.to_csv("NN-1.csv", index=True, index_label='Id')

In [ ]:
# Visualize the model results
plt.plot(clf.loss_curve_,'r')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.title("MCP Loss Evolution")
plt.grid()
plt.show()

In [ ]:
# Print the result of GridSearchCV(if used)
pd.DataFrame(clf.cv_results_).sort_values('rank_test_score').head(10)

In [ ]:
# Cross validation
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50,),max_iter= 120,verbose = True,random_state=1)
clf.fit(X_train, y_train)
# Model score
y_pred = clf.predict(X_test.fillna(0)) 
print("f1 score macro is :",f1_score(y_test, y_pred, average='macro'))
print("f1 score micro is :",f1_score(y_test, y_pred, average='micro'))
print("f1 score weighted is :",f1_score(y_test, y_pred, average='weighted'))

### XGBoost

In [ ]:
!pip install xgboost --upgrade
import xgboost as xgb

In [ ]:
parameter = {'n_estimators': [50, 100, 150]} 
             'learning_rate' : [0.05, 0.15, 0.30],
             'max_depth' : [5, 10, 20],
             'min_child_weight' : [ 1, 3, 7],
             'gamma': [0.0, 0.2, 0.4],
             'colsample_bytree' : [0.3, 0.5, 0.7]}

In [ ]:
xgb_model = xgb.XGBClassifier(n_jobs=1)
clf = GridSearchCV(xgb_model, parameter)
clf.fit(X, y)
print(clf.best_params_)

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values('rank_test_score').head(10)

In [ ]:
pred_y = clf.predict(test_df.fillna(0))
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("xgb.csv", index=True, index_label='Id')

### CatBoost

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor

In [ ]:
# Get the index of categorical features
cat_fea_idx = np.where(X.dtypes != np.float)[0]
cat_model = CatBoostClassifier(iterations=1000 ,
                               depth=4,
                               od_type="Iter",
                               early_stopping_rounds=500,
                               loss_function='MultiClass' )
cat_model.fit(X, y)
pred_y = cat_model.predict(test_df.fillna(0))
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("cat.csv", index=True, index_label='Id')

### Logistic Regression

In [ ]:
# Train logistic regression model
model = LogisticRegression(max_iter=500)
model.fit(training_features, y_train)

y_train_pred = model.predict(training_features)
accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train accuracy: {accuracy}")

# Predict labels for validation set and evaluate performance
y_val_pred = model.predict(valling_features)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy: {accuracy}")

# Predict labels for test set and generate submission file
y_test_pred = model.predict(testing_features)
submission = pd.DataFrame({"ID": range(test_data.shape[0]), "Predicted": y_test_pred})
submission.to_csv("submission.csv", index=False)

In [ ]:
# Hyperparameter tuning for LG(not quite help)
param_grid = [
    {'penalty' : ['l2','l1','none'],
    'C' : np.logspace(-4, 4, 10),
    'solver' : ['lbfgs','newton-cg'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

model = LogisticRegression()

from sklearn.model_selection import RandomizedSearchCV # Faster than Grid search
clf = RandomizedSearchCV(model, param_distributions = param_grid, cv = 3, verbose=True, n_jobs=-1)
best_clf = clf.fit(training_features, y_train)


In [ ]:
print (f'Accuracy - : {best_clf.score(training_features,y_train):.3f}')

### LightGBM

In [ ]:
# Empirical parameters from the internet
lgb_params = {
    'learning_rate': 0.007,
        'max_depth': 5,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
         'feature_fraction': 0.8,
         'num_leaves':30,
        'min_data_in_leaf':100,
        'lambda_l1': 0.8,
        'lambda_l2':0.9,
        'n_estimators':50}

In [ ]:
# Train with lightgbm
lgb_model = lgb.LGBMClassifier()
lgb_model.set_params(**lgb_params)
lgb_model.fit(training_features, y_train)

y_train_pred = lgb_model.predict(training_features)
accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train accuracy: {accuracy}")

# Predict labels for validation set and evaluate performance
y_val_pred = lgb_model.predict(valling_features)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy: {accuracy}")

# Predict labels for test set and generate submission file
y_test_pred = lgb_model.predict(testing_features)
submission = pd.DataFrame({"ID": range(test_data.shape[0]), "Predicted": y_test_pred})
submission.to_csv("lgb_submission.csv", index=False)


hyperparameter tuning

In [ ]:
# Hyperparameter tuning for LGBM(not very helpful at the end)
param_grid = [
    {'objetive' : ['binary'],
    'learning_rate' : [0.001,0.007,0.0001],
    'max_depth' : [1,3,5],
    'n_estimators': [50, 100,500, 1000],
    'lambda_l1': np.linspace(0,1,11),
    'lambda_l2': np.linspace(0,1,11)
    }
]

model = lgb.LGBMClassifier()

clf = RandomizedSearchCV(model, param_distributions = param_grid, cv = 5, verbose=True, n_jobs=-1)
best_clf = clf.fit(training_features, y_train)

In [ ]:
best_clf.best_estimator_

In [ ]:
print (f'Accuracy - : {best_clf.score(training_features,y_train):.3f}')

Model explanation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
feature_imp = pd.DataFrame(sorted(zip(lgb_model.feature_importances_,features)), columns=['Value','Feature']).sort_values(by='Value',ascending=False)

# Use the inherited function in lgbm to visualize the top 20 features
plt.figure(figsize=(6,6))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:20])
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

### Random Forest

In [ ]:
# Randomforest - overfitting, not used
rdf = RandomForestClassifier(n_estimators=50)
rdf.fit(training_features, y_train)
y_train_pred = rdf.predict(training_features)
accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train accuracy: {accuracy}")

# Predict labels for test set and generate submission file
y_test_pred = rdf.predict(testing_features)
submission = pd.DataFrame({"ID": range(test_data.shape[0]), "Predicted": y_test_pred})
submission.to_csv("rf_submission.csv", index=False)


### Model Evaluation

For classification tasks, the metrics are usually based on the confusion matrix and yield many derivatives according to the specific user cases.
Here are some examples:

Accuracy: Accuracy measures the proportion of correctly classified instances out of the total instances. It is suitable when the classes are balanced. However, it may not be the best metric for imbalanced datasets.
Use Case: Suitable for tasks like email spam detection or image recognition when the classes are roughly equal in size.

Precision: Precision measures the proportion of true positive predictions among all positive predictions. It is used to evaluate how well the model identifies true positive instances.
Use Case: Valuable when the cost of false positives is high, such as in medical diagnoses or fraud detection.

Recall (Sensitivity or True Positive Rate): Recall measures the proportion of true positive predictions among all actual positive instances. It assesses how well the model captures all positive instances.
Use Case: Important in scenarios where missing positive cases is costly, like disease detection or search and rescue operations.

F1-Score: The F1-Score is the harmonic mean of precision and recall, balancing both metrics. It is useful when you want to find a trade-off between precision and recall.
Use Case: Often used in situations where precision and recall are both important, such as information retrieval or recommendation systems.

ROC Curve and AUC: Receiver Operating Characteristic (ROC) curves plot the true positive rate against the false positive rate at various thresholds. The Area Under the Curve (AUC) summarizes the overall performance. It is robust to class imbalance.
Use Case: Suitable when you need to compare models' ability to discriminate between classes, e.g., in medical diagnosis or credit scoring.

In [ ]:
# For classification task Confusion matrix and several related metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Create heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt=".2%", cmap="Reds")

# Get class labels from indexes using the idx_to_class dictionary
class_labels = [id2label[str(label_idx)] for label_idx in sorted(set(true_labels))]

plt.title("Confusion Matrix (Accuracy: {:.2%})".format(accuracy))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")

# Set x-axis and y-axis labels using class labels
plt.xticks(np.arange(len(class_labels)) + 0.5, class_labels)
plt.yticks(np.arange(len(class_labels))+ 0.5, class_labels)

plt.show()

print("Accuracy: {:.2%}".format(accuracy))
print("F1 Score: {:.2%}".format(f1))
print("Precision: {:.2%}".format(precision))
print("Recall: {:.2%}".format(recall))


In [ ]:
import matplotlib.ticker as mticker

# Count the occurrences of each class in false_pred
class_counts = {}
for label in false_pred:
    if label in class_counts:
        class_counts[label] += 1
    else:
        class_counts[label] = 1

# Calculate the total count
total_count = sum(class_counts.values())

# Calculate the percentage for each class
class_labels = [id2label[str(label)] for label in class_counts.keys()]
class_percentages = [(count / total_count) * 100 for count in class_counts.values()]

# Create a bar plot of class proportions
plt.figure(figsize=(8, 6))
plt.bar(class_labels, class_percentages)
plt.xlabel('Class Labels')
plt.ylabel('Percentage')
plt.title('Proportion of Each Class in False Predictions')

# Add percentage sign to y-axis labels
plt.gca().yaxis.set_major_formatter(mticker.PercentFormatter())

plt.show()

In [ ]:
# Plotting ROC Curve
%matplotlib inline
import matplotlib.pyplot as plt

y_pred_prob = logreg.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for xx classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

## Regression Tasks

In fact, several models shown above used for classification task have their regressor too, I don't repeat here.

### LightGBM(Model Interpretation)

In [ ]:
# Model Interpretaion
! pip install shap

In [ ]:
import shap
shap.initjs()

In [ ]:
# Instantiation of a LGBMRegressor model

lgbm_reg = LGBMRegressor(n_estimators = 200, learning_rate = 0.07, n_jobs = -1, random_state = 42)
lgbm_reg.fit(X_train_scaled, y_train)

# Predict log_SalePrice for the variables in the training set
y_pred_lgbm_train = lgbm_reg.predict(X_train_scaled)

rmse_lgbm_train = np.sqrt(metrics.mean_squared_error(y_train, y_pred_lgbm_train))
r2_lgbm_train = metrics.r2_score(y_train, y_pred_lgbm_train)
print("Training RMSE:", rmse_lgbm_train)
print("Training R²:", r2_lgbm_train)

print("\n")

# Predict log_SalePrice for the variables in the validation set
y_pred_lgbm = lgbm_reg.predict(X_val_scaled)

rmse_lgbm = np.sqrt(metrics.mean_squared_error(y_val, y_pred_lgbm))
r2_lgbm = metrics.r2_score(y_val, y_pred_lgbm)
print("Out-of-sample RMSE:", rmse_lgbm)
print("Out-of-sample R²:", r2_lgbm)

Model explanation

In [ ]:
# Print feature importance
lgb.plot_importance(lgbm_reg)
plt.show()

In [ ]:
# Save the model result for later use
dump(lgbm_reg, 'lgbm_reg.joblib')

In [ ]:
# Load the pretrained model
loaded_model = load('lgbm_reg.joblib')

In [ ]:
# Plot one of the decision tree to see how the criteria work
lightgbm.plot_tree(lgbm_reg,tree_index=0,figsize=(100,50))

In [ ]:
custom_labels = ['', '', '']

In [ ]:
# Try to use SHAP value to interprete the model
explainer = shap.Explainer(lgbm_reg)
shap_values = explainer(X_train_scaled)

shap.summary_plot(shap_values, X_train_scaled, plot_type = "bar",feature_names=custom_labels)
shap.plots.bar(shap_values)

In [ ]:
# The impact of the feature on model output
shap.summary_plot(shap_values, X_train_scaled,feature_names=custom_labels)

In [ ]:
# Plot the heatmap for each instance
shap_values_explaination = shap.Explanation(shap_values, feature_names=custom_labels) 
shap.plots.heatmap(shap_values_explaination)

In [ ]:
# Change the column name
shap_values.feature_names = custom_labels

In [ ]:
# Reverse the scaler
shap_values.data = scaler.inverse_transform(shap_values.data)

In [ ]:
# Plot only one instance
shap.plots.waterfall(shap_values[0])

### Model evaluation

Mean Absolute Error (MAE): MAE measures the average absolute difference between predicted and actual values. It gives equal weight to all errors.
Use Case: Suitable for tasks like predicting house prices, where you want to quantify the average magnitude of errors.

Mean Squared Error (MSE): MSE calculates the average of the squared differences between predicted and actual values. It penalizes larger errors more heavily.
Use Case: Commonly used for regression problems like stock price prediction or weather forecasting.

Root Mean Squared Error (RMSE): RMSE is the square root of MSE, providing an interpretable metric in the same units as the target variable.
Use Case: Useful when you want to express the model's error in a more interpretable way, such as predicting product sales.

R-squared (Coefficient of Determination): R-squared measures the proportion of the variance in the dependent variable that is explained by the independent variables. It ranges from 0 to 1, with higher values indicating better model fit.
Use Case: Often used to assess the goodness-of-fit of regression models, e.g., in economic forecasting or scientific research.

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Mean Absolute Error (MAE)
mae = mean_absolute_error(actual, predicted)
print("Mean Absolute Error:", mae)

# Mean Squared Error (MSE)
mse = mean_squared_error(actual, predicted)
print("Mean Squared Error:", mse)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R-squared (R2)
r2 = r2_score(actual, predicted)
print("R-squared (R2):", r2)